[Cleaning](clean.ipynb)

# EDA

Import and define some functions

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from os.path import exists
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

sns.set_theme(style='darkgrid')

df = pd.read_csv('data/clean.csv')
y = df.mpg

def show_plots(filenames):
    for j in range(0,len(filenames),2):
        if (len(filenames)-j)>1:
            display(Markdown(f'![]({filenames[j]})![]({filenames[j+1]})'))
        else:
            display(Markdown(f'![]({filenames[j]})'))

def make_plots(df, y):
    filenames = []
    
    for col in df.columns:
        filename = 'img/%s_joint.png' % col
        filenames.append(filename)
        if not exists(filename):
            sns.jointplot(x=df[col],y=y,kind='reg',
                          joint_kws={'scatter_kws':dict(alpha=0.3)})
            plt.suptitle(f'{col} vs mpg')
            plt.subplots_adjust(top=.93)
            plt.savefig(filename,facecolor='white',transparent=False)
            plt.close()
        
    show_plots(filenames)

So I'm only interested in a few things:

In [2]:
make_plots(df[['cylinders','displacement',
            'horsepower','weight',]],y)

![](img/cylinders_joint.png)![](img/displacement_joint.png)

![](img/horsepower_joint.png)![](img/weight_joint.png)

As the number of cylinders, displacement, horsepower, or weight increase, MPG goes down.

I want to know more:

In [3]:
new_features = pd.DataFrame()
new_features['efficiency'] = df.horsepower / df.displacement
new_features['load'] = df.displacement / df.weight
new_features['bore_size'] = df.displacement / df.cylinders
new_features['grunt'] = new_features.bore_size / new_features.efficiency

In [4]:
merged = df.join(new_features)
del df

Check Correlation

In [5]:
merged.corrwith(y).sort_values()

weight         -0.831745
displacement   -0.804456
horsepower     -0.777897
cylinders      -0.776090
bore_size      -0.773403
load           -0.714996
grunt          -0.712074
acceleration    0.420414
efficiency      0.509309
origin          0.563833
model_year      0.580091
mpg             1.000000
dtype: float64

In [6]:
make_plots(new_features,y)
del new_features

![](img/efficiency_joint.png)![](img/load_joint.png)

![](img/bore_size_joint.png)![](img/grunt_joint.png)

* HP per cubic inch is a measure of engine efficiency, as this increases so does MPG
* Load is a metric of how hard the engine has to work compared to its size. Engines that work hard use more fuel and a small engine working really hard can use more fuel than a big engine not doing much
* Bore_size is an attempt to describe cylinder bore diameter which gives insight on torque curve
* Grunt is an attempt to describe the power curve of an engine, or more specifically the presence/absence of low rpm torque output

## A bit on engines:

* A most basic description of an engine is that it's an air pump
* Horsepower = (Torque * RPM) / 5252
* Torque peak is where an engine is operating most efficiently as far as air flow, applied science in action. (Fluid dynamics, resonance)
* Operating above or below the torque peak reduces efficiency and efficiency == fuel economy
* Torque peaks normally occur below 5252rpm, and horsepower peaks above that, so long as the engine can actually rev that high. On a dyno sheet (measuring torque and horsepower vs rpm) you'll see the torque/horsepower lines cross at 5252rpm
* As an engine spins faster, the power output increases until combustion is so inefficient and it produces so little torque that spinning faster produces no more power, if it holds together that long

Basically an engine that makes lots of power at high rpm but relatively little low end torque (mazda rotary), is going to have poor fuel economy because it spends most of its time outside of its efficiency range. In contrast, diesel engines typically turn lower rpms and create all kinds of torque down low. So not only do they start off making more torque but they are less likely to stray very far from torque peak. This is also why horsepower numbers on a diesel appear low, because they can't rev as high. There's more to it than this but this should be enough to provide context.

## Some stats

In [7]:
print(f'''Mean MPG: {y.mean():.2f}
Mean Weight: {merged.weight.mean():.2f}
Mean Horsepower: {merged.horsepower.mean():.2f}''')

for col in merged.columns[9:]:
    print(f'{col} mean: {merged[col].mean():.2f}')

Mean MPG: 23.51
Mean Weight: 2970.59
Mean Horsepower: 104.12
efficiency mean: 0.61
load mean: 0.06
bore_size mean: 33.36
grunt mean: 62.78


### What's all that on the edges?

In [8]:
merged[merged.efficiency>1]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,efficiency,load,bore_size,grunt
71,19.0,3,70.0,97.0,2330.0,13.5,72,3,mazda rx2 coupe,1.385714,0.030043,23.333333,16.838488
111,18.0,3,70.0,90.0,2124.0,13.5,73,3,maxda rx3,1.285714,0.032957,23.333333,18.148148
243,21.5,3,80.0,110.0,2720.0,13.5,77,3,mazda rx-4,1.375000,0.029412,26.666667,19.393939
334,23.7,3,70.0,100.0,2420.0,12.5,80,3,mazda rx-7 gs,1.428571,0.028926,23.333333,16.333333


These are the Mazda rotaries, otherwise known as [Wankel Engines](https://en.wikipedia.org/wiki/Wankel_engine)

Efficient power for their size because they can rev to 7000rpm or so, and that's where they make peak power. Not good for fuel economy. Note the low gruntiness

In [9]:
merged[merged.cylinders==5]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,efficiency,load,bore_size,grunt
274,20.3,5,131.0,103.0,2830.0,15.9,78,2,audi 5000,0.786260,0.046290,26.2,33.322330
297,25.4,5,183.0,77.0,3530.0,20.1,79,2,mercedes benz 300d,0.420765,0.051841,36.6,86.984416
327,36.4,5,121.0,67.0,2950.0,19.9,80,2,audi 5000s (diesel),0.553719,0.041017,24.2,43.704478


Look at the gruntiness and mpg of these diesels! For comparison the first Audi appears to be a gas engine. Consider the displacement and power. The one below as well

In [10]:
merged.iloc[np.where((merged.mpg>35) & (merged.displacement > 250))]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,efficiency,load,bore_size,grunt
386,38.0,6,262.0,85.0,3015.0,17.0,82,1,oldsmobile cutlass ciera (diesel),0.324427,0.086899,43.666667,134.596078


This is an interesting engine. In fact, [these cars are rumored to be the reason why diesel cars are so unpopular in North America](https://www.autotrader.com/car-news/when-diesel-was-dreadful-oldsmobile-diesels-259997). [Here is a more technical write-up](https://www.dieselworldmag.com/diesel-engines/oldsmobile-350-v8)

But that's a bit beside the point, the engines above and below for sake of conversation are basically the same, the V6 being the same as the V8 but with 2 less cylinders. So compare the stats between them as gas and diesel


In [11]:
merged.iloc[np.where((merged.mpg>20) & (merged.displacement > 340))]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,efficiency,load,bore_size,grunt
298,23.0,8,350.0,125.0,3900.0,17.4,79,1,cadillac eldorado,0.357143,0.089744,43.75,122.500000
363,26.6,8,350.0,105.0,3725.0,19.0,81,1,oldsmobile cutlass ls,0.300000,0.093960,43.75,145.833333


Big lazy engines in big heavy cars don't have to have poor MPG!

In [12]:
merged[merged.load>0.14]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,efficiency,load,bore_size,grunt
13,14.0,8,455.0,225.0,3086.0,10.0,70,1,buick estate wagon (sw),0.494505,0.14744,56.875,115.013889


Big car, big engine, terrible MPG.. That weight is way off

In [13]:
merged.at[13,'weight'] = 5000
merged['load']  = merged.displacement / merged.weight

What vehicles have the lowest MPG?

In [14]:
merged.sort_values('mpg').head(25)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,efficiency,load,bore_size,grunt
28,9.0,8,304.0,193.0,4732.0,18.5,70,1,hi 1200d,0.634868,0.064243,38.000,59.854922
26,10.0,8,307.0,200.0,4376.0,15.0,70,1,chevy c20,0.651466,0.070155,38.375,58.905625
25,10.0,8,360.0,215.0,4615.0,14.0,70,1,ford f250,0.597222,0.078007,45.000,75.348837
27,11.0,8,318.0,210.0,4382.0,13.5,70,1,dodge d200,0.660377,0.072570,39.750,60.192857
103,11.0,8,400.0,150.0,4997.0,14.0,73,1,chevrolet impala,0.375000,0.080048,50.000,133.333333
67,11.0,8,429.0,208.0,4633.0,11.0,72,1,mercury marquis,0.484848,0.092597,53.625,110.601562
124,11.0,8,350.0,180.0,3664.0,11.0,73,1,oldsmobile omega,0.514286,0.095524,43.750,85.069444
42,12.0,8,383.0,180.0,4955.0,11.5,71,1,dodge monaco (sw),0.469974,0.077296,47.875,101.867361
95,12.0,8,455.0,225.0,4951.0,11.0,73,1,buick electra 225 custom,0.494505,0.091901,56.875,115.013889
90,12.0,8,429.0,198.0,4952.0,11.5,73,1,mercury marquis brougham,0.461538,0.086632,53.625,116.187500


What vehicles have the Highest MPG?

In [15]:
merged.sort_values('mpg',ascending=False).head(25)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,efficiency,load,bore_size,grunt
322,46.6,4,86.0,65.0,2110.0,17.9,80,3,mazda glc,0.755814,0.040758,21.500000,28.446154
329,44.6,4,91.0,67.0,1850.0,13.8,80,3,honda civic 1500 gl,0.736264,0.049189,22.750000,30.899254
325,44.3,4,90.0,48.0,2085.0,21.7,80,2,vw rabbit c (diesel),0.533333,0.043165,22.500000,42.187500
393,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup,0.536082,0.045540,24.250000,45.235577
326,43.4,4,90.0,48.0,2335.0,23.7,80,2,vw dasher (diesel),0.533333,0.038544,22.500000,42.187500
244,43.1,4,90.0,48.0,1985.0,21.5,78,2,volkswagen rabbit custom diesel,0.533333,0.045340,22.500000,42.187500
309,41.5,4,98.0,76.0,2144.0,14.7,80,2,vw rabbit,0.775510,0.045709,24.500000,31.592105
330,40.9,4,85.0,53.5,1835.0,17.3,80,2,renault lecar deluxe,0.629412,0.046322,21.250000,33.761682
324,40.8,4,85.0,65.0,2110.0,19.2,80,3,datsun 210,0.764706,0.040284,21.250000,27.788462
247,39.4,4,85.0,70.0,2070.0,18.6,78,3,datsun b210 gx,0.823529,0.041063,21.250000,25.803571


Pick a few to toss into the model and get some numbers out

In [16]:
merged.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model_year', 'origin', 'car_name', 'efficiency',
       'load', 'bore_size', 'grunt'],
      dtype='object')

In [17]:
X = merged[[\
        'horsepower', # overall power
        'bore_size', # "torque curve"
        'grunt',
        'load',  # load
       ]]

X.to_csv('data/X.csv',index=False)
y.to_csv('data/y.csv',index=False)

[Modeling](model.ipynb)